`s4rdm3x` requires packages relationships to be defined in a `.sysmdl` file. [`jdeps`](https://docs.oracle.com/javase/8/docs/technotes/tools/unix/jdeps.html) is available in JDK 15, and can be used to help automating this step.

In [1]:
!~/Development/java/jdk-15/bin/jdeps ~/Downloads/1.3/data/systems/commons-imaging/commons-imaging-1.0-alpha2.jar | head -n 10

commons-imaging-1.0-alpha2.jar -> java.base
commons-imaging-1.0-alpha2.jar -> java.desktop
commons-imaging-1.0-alpha2.jar -> java.logging
   org.apache.commons.imaging                         -> java.awt                                           java.desktop
   org.apache.commons.imaging                         -> java.awt.color                                     java.desktop
   org.apache.commons.imaging                         -> java.awt.image                                     java.desktop
   org.apache.commons.imaging                         -> java.io                                            java.base
   org.apache.commons.imaging                         -> java.lang                                          java.base
   org.apache.commons.imaging                         -> java.util                                          java.base
   org.apache.commons.imaging                         -> java.util.logging                                  java.logging


In [2]:
ref_table  = {
    'color': 'org.apache.commons.imaging.color.*',
    'common': 'org.apache.commons.imaging.common.*',
    'bmp': 'org.apache.commons.imaging.formats.bmp.*',
    'dcx': 'org.apache.commons.imaging.formats.dcx.*',
    'gif': 'org.apache.commons.imaging.formats.gif.*',
    'icns': 'org.apache.commons.imaging.formats.icns.*',
    'ico': 'org.apache.commons.imaging.formats.ico.*',
    'jpeg': 'org.apache.commons.imaging.formats.jpeg.*',
    'pcx': 'org.apache.commons.imaging.formats.pcx.*',
    'png': 'org.apache.commons.imaging.formats.png.*',
    'pnm': 'org.apache.commons.imaging.formats.pnm.*',
    'psd': 'org.apache.commons.imaging.formats.psd.*',
    'rgbe': 'org.apache.commons.imaging.formats.rgbe.*',
    'tiff': 'org.apache.commons.imaging.formats.tiff.*',
    'wbmp': 'org.apache.commons.imaging.formats.wbmp.*',
    'xbm': 'org.apache.commons.imaging.formats.xbm.*',
    'xpm': 'org.apache.commons.imaging.formats.xpm.*',
    'icc': 'org.apache.commons.imaging.formats.icc.*',
    'internal': 'org.apache.commons.imaging.formats.internal.*',
    'palette': 'org.apache.commons.imaging.formats.palette.*',
    'base': 'org.apache.commons.imaging(?!(.common|.formats|.internal|.palette)).*'
}

In [3]:
# store the output of the shell command
import os
cmd = """
~/Development/java/jdk-15/bin/jdeps ~/Downloads/1.3/data/systems/commons-imaging/commons-imaging-1.0-alpha2.jar | sed 's/^  *//;s/ *$//' | grep "commons-imaging-1.0-alpha2.jar" | grep "org.apache.commons.imaging" | awk '{print $1 " " $3}'
"""
lines = os.popen(cmd).read()
lines[0:100]

'org.apache.commons.imaging org.apache.commons.imaging.common\norg.apache.commons.imaging org.apache.c'

Now replace the package names by their aliases, using `ref_table`.

In [4]:
import re
from functools import lru_cache

@lru_cache
def get_module_alias(m):
    for alias, package_regex in ref_table.items():
        regex = re.compile(package_regex)
        if regex.match(source_module):
            return alias

entries = []
        
for l in [x for x in lines.split('\n') if x != '']:
    source_module, target_module = l.split(' ')
    s = get_module_alias(source_module)
    t = get_module_alias(target_module)
    token = f'{s} {t}'
    if token not in entries:
        entries.append(token)

In [5]:
entries.sort()
for e in entries:
    print(e)

base base
base bmp
base gif
base jpeg
base png
base psd
base tiff
bmp base
bmp jpeg
common base
jpeg base
jpeg common
jpeg gif
jpeg jpeg
jpeg tiff
png base
psd base
psd gif
psd jpeg
psd psd
tiff base
tiff common
tiff gif
tiff jpeg
tiff tiff
